## <font color='blue'>SME0104 - Cálculo Numérico</font>

### Primeiro Trabalho em Grupo 
#### Comparação de Métodos na Solução do Laplaciano em Grafos para propagação de informação

**Luis Gustavo Nonato**

------

Considere os arquivos `manh.el` e `manh.xy` que fornecem as arestas e as coordenadas dos vértices do grafo de ruas da ilha de Manhattan, NY (arquivos disponíveis para download no Google Drive).

O grafo de ruas possui diversas componentes conexas, considerando somente a maior componente conexa, você deve realizar as seguintes tarefas:
1. Selecione alguns vértices do grafo $v_{i_1},v_{i_2},\ldots,v_{i_k},\, k<<n$ ($n$ é o número de vértices na maior componente do grafo e $k$ é um número bem menor que $n$, $k=10$ por exemplo) e atribua valores distindos $c_{i_1},c_{i_2},\ldots,c_{i_k}$ a cada um dos vértices selecionados (por exemplo valores no intervalo (0,10]);
2. Construa a matriz Laplaciana $L$ do grafo de ruas;
3. Construa a matriz de penalidades $P$, sendo $P$ é uma matriz diagonal onde a entrada $P_{jj}=\alpha$ se $j$ corresponde ao índice de algum dos vértices escolhidos no item 1 acima ($\alpha=1.0e7$ por exemplo), sendo $P_{ii}=0$ caso contrário.
4. Construa um vetor $b$ da seguinte forma:
$$
b_{j} = \left\{\begin{matrix} c_{i_s} & \text{ se } j = i_s \\ 0 & \text{ caso contrário}\end{matrix}\right.
$$
5. Compare o tempo de solução do sistema
$$
(L+P)x = Px
$$
para os métodos:
    - Decomposição LU
    - Cholesky
    - Jacobi e Gaus-Seidel
    - Gradientes Conjugados
    
6. Refaça as tarefas com representação por matriz esparsa e matrizes cheias, comparando os resultados.

In [37]:
# Solucao
import numpy as np
import networkx as nx
import scipy as sp

def gauss_seidel(A, b, tolerance, max_iterations, x):
    #x is the initial condition
    iter1 = 0
    #Iterate
    for k in range(max_iterations):
        iter1 = iter1 + 1
        print ("The solution vector in iteration", iter1, "is:", x)    
        x_old  = x.copy()
        
        #Loop over rows
        for i in range(A.shape[0]):
            x[i] = (b[i] - np.dot(A[i,:i], x[:i]) - np.dot(A[i,(i+1):], x_old[(i+1):])) / A[i ,i]
            
        #Stop condition 
        #LnormInf corresponds to the absolute value of the greatest element of the vector.
        
        LnormInf = max(abs((x - x_old)))/max(abs(x_old))   
        print ("The L infinity norm in iteration", iter1,"is:", LnormInf)
        if  LnormInf < tolerance:
            break
    
           
    return x

def jacobi(A,b,x0,tol,N):  
    #preliminares  
    A = A.astype('double')  
    b = b.astype('double')  
    x0 = x0.astype('double')  
 
    n= np.shape(A)[0]  
    x = np.zeros(n)  
    it = 0  
    #iteracoes  
    while (it < N):  
        it = it+1  
        #iteracao de Jacobi  
        for i in np.arange(n):  
            x[i] = b[i]  
            for j in np.concatenate((np.arange(0,i),np.arange(i+1,n))):  
                x[i] -= A[i,j]*x0[j]  
            x[i] /= A[i,i]  
        #tolerancia  
        if (np.linalg.norm(x-x0,np.inf) < tol):  
            return x  
        #prepara nova iteracao  
        x0 = np.copy(x)  
    raise NameError('num. max. de iteracoes excedido.')


In [34]:


G = nx.read_edgelist("./dados/manh.el", create_using=nx.Graph, nodetype=int)

componentes = [G.subgraph(c).copy() for c in nx.connected_components(G)]
# nx.draw_networkx(componentes[0],node_size=10,with_labels=False)

number_of_nodes = []
for i in range(len(componentes)):
  number_of_nodes.append(componentes[i].number_of_nodes())

index_maior_componente = number_of_nodes.index(max(number_of_nodes))
grafo_maior_componente = componentes[index_maior_componente]

matriz_laplaciana = nx.laplacian_matrix(grafo_maior_componente).todense()
matriz_penalidade = np.zeros( (len(matriz_laplaciana), len(matriz_laplaciana)))

penalidade = 10**7
for i in range(11):
  matriz_penalidade[i][i] = penalidade

b = np.zeros((len(matriz_penalidade),1))

b[0][0] = 25
b[1][0] = 4
b[2][0] = 1
b[3][0] = 27
b[4][0] = 38
b[5][0] = 39
b[6][0] = 7
b[7][0] = 40
b[8][0] = 13
b[9][0] = 28
b[10][0] = 20

A = np.add(matriz_laplaciana,matriz_penalidade)

# lu
lu, piv = sp.linalg.lu_factor(A)
x_lu = sp.linalg.lu_solve((lu,piv),b)

# chow chow
c, low = sp.linalg.cho_factor(A)
x_cho = sp.linalg.cho_solve((c,low),b)



In [38]:
xxx = np.zeros((len(matriz_penalidade),1))
aa = gauss_seidel(A,b,10**(-6),10**5,xxx)

The solution vector in iteration 1 is: [[0.]
 [0.]
 [0.]
 ...
 [0.]
 [0.]
 [0.]]


/tmp/ipykernel_73886/547644433.py:22: RuntimeWarning: divide by zero encountered in true_divide
  LnormInf = max(abs((x - x_old)))/max(abs(x_old))


The L infinity norm in iteration 1 is: [inf]
The solution vector in iteration 2 is: [[2.49999925e-06]
 [4.00000130e-07]
 [1.00000260e-07]
 ...
 [0.00000000e+00]
 [0.00000000e+00]
 [0.00000000e+00]]
The L infinity norm in iteration 2 is: [0.28437497]
The solution vector in iteration 3 is: [[2.49999957e-06]
 [4.00000153e-07]
 [1.00000263e-07]
 ...
 [0.00000000e+00]
 [0.00000000e+00]
 [0.00000000e+00]]
The L infinity norm in iteration 3 is: [0.1187717]
The solution vector in iteration 4 is: [[2.49999957e-06]
 [4.00000157e-07]
 [1.00000266e-07]
 ...
 [2.01066295e-13]
 [0.00000000e+00]
 [0.00000000e+00]]
The L infinity norm in iteration 4 is: [0.0589551]
The solution vector in iteration 5 is: [[2.49999957e-06]
 [4.00000159e-07]
 [1.00000267e-07]
 ...
 [1.44305707e-12]
 [0.00000000e+00]
 [0.00000000e+00]]
The L infinity norm in iteration 5 is: [0.03902449]
The solution vector in iteration 6 is: [[2.49999957e-06]
 [4.00000160e-07]
 [1.00000268e-07]
 ...
 [5.69693388e-12]
 [0.00000000e+00]
 [0

KeyboardInterrupt: 